In [ ]:
green_areas = r'G:/otol/exp2_evu/ref/cos18_forevu.shp'

gid = 'ffif'

buffer_dist = 250

urban_areas = r'G:\otol\guavsforest.gdb\osm\urban50m'

tmpws = r'G:\otol\guavsforest.gdb\corre_cos_s1o'

epsg = 3763

outfeat = r'G:\otol\guavsforest.gdb\cosres\cosres_urbanosm'

bname = 'coss1o'

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [ ]:
srs = "PROJCS[\"ETRS_1989_Portugal_TM06\",GEOGCS[\"GCS_ETRS_1989\",DATUM[\"D_ETRS_1989\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-8.133108333333334],PARAMETER[\"Scale_Factor\",1.0],PARAMETER[\"Latitude_Of_Origin\",39.66825833333333],UNIT[\"Meter\",1.0]]"

In [ ]:
import arcpy
import os

from gesri.rd.shp import shp_to_lyr
from glass.pys.oss import fprop
from glass.gp.prox.bfing import cheese_buffer
from gesri.dp       import copy_feat
from gesri.tbl.attr import geom_attr_to_shp
from gesri.tbl.col import add_col, cols_calc
from gesri.tbl.joins import join_table
from gesri.gp.ovl import intersection
from gesri.gp.gen import dissolve

In [ ]:
arcpy.env.overwriteOutput = True

In [ ]:
# Get Cheese buffer

cheese_bf = cheese_buffer(
    green_areas, buffer_dist, 45,
    os.path.join(os.path.dirname(green_areas), f"cheese_{fprop(green_areas, 'fn')}.shp"),
    uniqueid='jpk', areaf="area_o",
    epsg=epsg, shpid=gid
)

cheese_lyr = shp_to_lyr(cheese_bf)

In [ ]:
# Intersect cheese buffer with urban regions

urban_lyr = shp_to_lyr(urban_areas)

ifeat = os.path.join(tmpws, f'{bname}_uc')

ilyr = intersection([urban_lyr, cheese_lyr], ifeat)

idiss = os.path.join(tmpws, f'{bname}_ucdiss')

idiss_lyr = dissolve(ilyr, idiss, [gid, "direction"])

In [ ]:
# Get intersection area
idiss_lyr = geom_attr_to_shp(
    idiss_lyr, "areaint",
    geom_attr="AREA", srs=srs,
    area_unit="SQUARE_METERS"
)

# Get Feature unique identifier
idiss_lyr = add_col(idiss_lyr, "jfk", "TEXT", 30)

idiss_lyr = cols_calc(idiss_lyr, "jfk", f"str(int(!{gid}!)) + '__' + !direction!")

In [ ]:
# Join to obtain intersected area in the original feature class
cheese_lyr = join_table(
    cheese_lyr, idiss_lyr,
    "jpk", "jfk",
    cols=["areaint"]
)

In [ ]:
# Get Urban intersection by each buffer slice
#cheese_lyr = shp_to_lyr(cheese_bf, f"{fprop(heese_bf, 'fn')}_l2")

cheese_lyr = add_col(cheese_lyr, "urbanp", "DOUBLE", None)

cheese_lyr = cols_calc(cheese_lyr, "urbanp", "!areaint! / !area_o! * 100")

In [ ]:
# Get buffer statistics
cheese_lyr = add_col(cheese_lyr, "cdir", "LONG", None)

cheese_lyr = cols_calc(cheese_lyr, "cdir", "is_dir(!urbanp!)", code="""def is_dir(f):
    if f > 0:
        return 1
    else:
        return 0""")

green_buffer = os.path.join(tmpws, f'{bname}_greenbf')
lyr_greenbf = dissolve(
    cheese_lyr, green_buffer, gid,
    statistics=[["cdir", "SUM"], ["areaint", "SUM"], ["area_o", "SUM"]]
)

In [ ]:
# Get Urban percentage in the buffer

lyr_greenbf = add_col(lyr_greenbf, "p_edi_buf", "DOUBLE", None)

lyr_greenbf = cols_calc(lyr_greenbf, "p_edi_buf", "calc_area(!SUM_area_o!,!SUM_areaint!)", code="""def calc_area(total, areai):
    if areai == None:
        return 0
    else:
        return areai / total * 100"""
)

In [ ]:
# Copy initial shape
cp_green = copy_feat(green_areas, outfeat)

# Join with initial shape
green_lyr = join_table(
    cp_green, lyr_greenbf,
    gid, gid,
    cols=["SUM_cdir", "p_edi_buf", "SUM_areaint"]
)

In [ ]:
# Get intersected area propotion in relation
# to orignal polygon area
green_lyr = geom_attr_to_shp(
    green_lyr, "garea",
    geom_attr="AREA", srs=srs,
    area_unit="SQUARE_METERS"
)

green_lyr = add_col(green_lyr, "p_edi", "DOUBLE", None)

green_lyr = cols_calc(green_lyr, "p_edi", "calc_area(!garea!,!SUM_areaint!)", code="""def calc_area(total, areai):
    if areai == None:
        return 0
    else:
        return areai / total"""
)

In [ ]:
green_lyr = add_col(green_lyr, "uareahec", "DOUBLE", None)

green_lyr = cols_calc(green_lyr, "uareahec", "calc_area(!SUM_areaint!)", code="""def calc_area(areai):
    if areai == None:
        return 0
    else:
        return areai / 10000"""
)